In [1]:
!python -V

Python 3.12.1


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
df = pd.read_parquet('C:\\Users\\Lotos\\Downloads\\yellow_tripdata_2023-01.parquet')
print(df.head(3))

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
num_columns = len(df.columns)
print("Number of columns:", num_columns)

Number of columns: 19


In [ ]:
# Drop outliers
df_original = df.copy()
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Calculate the fraction of records left
fraction_remaining = df.shape[0] / len(df_original)
print(f"Fraction of records left: {fraction_remaining}")



Fraction of records left: 0.9812202822125979


In [ ]:
from sklearn.feature_extraction import DictVectorizer

# Convert dataframe to a list of dictionaries
dict_list = df[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Fit a dictionary vectorizer with sparse output
vectorizer = DictVectorizer(sparse=True)
feature_matrix = vectorizer.fit_transform(dict_list)

# Get the dimensionality of the feature matrix
num_columns = feature_matrix.shape[1]

print("Dimensionality of the feature matrix:", num_columns)


Dimensionality of the feature matrix: 515


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Train-test split
X_train = feature_matrix
y_train = df['duration']

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the training data
y_train_pred = model.predict(X_train)

# Calculate RMSE on training data
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

print("RMSE on train:", rmse_train)


 

RMSE on train: 7.649262060255514


In [ ]:
df_val = pd.read_parquet('C:\\Users\\Lotos\\Downloads\\yellow_tripdata_2023-02.parquet')
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]
# Convert validation dataframe to a list of dictionaries
dict_list_val = df_val[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Transform validation data using the same vectorizer
X_val = vectorizer.transform(dict_list_val)

# Make predictions on the validation data
y_val_pred = model.predict(X_val)

# Calculate RMSE on validation data
y_val = df_val['duration']
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))

print("RMSE on validation:", rmse_val)


RMSE on validation: 7.811816183354732
